In [34]:
import sys
import os

# Add the simulations/mcpy directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../simulations')))

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from nnpiv.linear import sparse_l1vsl1, sparse_ridge_l1vsl1, sparse2_l1vsl1, sparse2_ridge_l1vsl1
from sklearn.preprocessing import PolynomialFeatures

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import os
import numpy as np
import simulations.dgps_mediated as dgps
import sys

# Now you can import the module
from nnpiv.semiparametrics import DML_mediated, DML_joint_mediated

In [35]:
deg = 1
fn_number = 0
tau_fn = dgps.get_tau_fn(fn_number)
tauinv_fn = dgps.get_tauinv_fn(fn_number)
W, Z, X, M, D, Y, tau_fn = dgps.get_data(2000, tau_fn)

V = np.random.rand(Y.shape[0])
V = V.reshape(-1, 1)

print(np.column_stack((W,X,Z)).shape)
ind = np.where(D==0)[0]
W0 = W[ind]
X0 = X[ind,:]
W0_test = np.zeros((1000, 1+X.shape[1]))
W0_test += np.median(np.column_stack((X0,W0)), axis=0, keepdims=True)
W0_test[:, 2] = np.linspace(np.percentile(
            W0[:, 0], 5), np.percentile(W0[:, 0], 95), 1000)

# True parameters
b_yd = 2.0; b_ym = 1.0; b_yx = np.array([[-1.0],[-1.0]]); b_yu = -1.0; b_yw = 2.0; b_y0 = 2.0
b_wx = np.array([[0.2],[0.2]]); b_wu = -0.6; b_w0 = 0.3
b_md = -0.3; b_mx = np.array([[-0.5],[-0.5]]); b_mu = 0.4; b_m0 = 0.0
    
gamma_1w = (b_yw*b_wu + b_yu)/b_wu
gamma_1x = b_yw*b_wx + b_yx - gamma_1w*b_wx
gamma_1m = b_ym
gamma_10 = b_y0 + b_yd + b_yw*b_w0 - gamma_1w*b_w0

gamma_0w = (gamma_1m*b_mu + gamma_1w*b_wu)/b_wu
gamma_0x = gamma_1m*b_mx + gamma_1w*b_wx + gamma_1x - gamma_0w*b_wx
gamma_00 = gamma_10 + gamma_1m*b_m0 + gamma_1w*b_w0 - gamma_0w*b_w0

    # True nuisance function
expected_te = gamma_00 + tauinv_fn(W0_test)@np.row_stack((gamma_0x, gamma_0w))
D_ = D.copy()


(2000, 4)


In [36]:
splin_1 = sparse_l1vsl1(B=100, lambda_theta=.1,
                                             eta_theta=.1,
                                             eta_w=.1,
                                             n_iter=10000, tol=.0001, sparsity=None)

trans = PolynomialFeatures(degree=deg, include_bias=False)


In [37]:

dml_splin = DML_mediated(Y, D, M, W, Z, X,
                        estimator='MR',
                        estimand='E[Y(1,M(0))]',
                        model1 = splin_1,
                        model2 = splin_1,
                        modelq1 = splin_1,
                        modelq2 = splin_1,
                n_folds=5, n_rep=1, opts={'lin_degree': deg})
print(dml_splin.dml())


Rep: 1


100%|██████████| 5/5 [00:24<00:00,  4.91s/it]

(4.2837892525237455, 19.267768278245764, array([4.09141418, 4.47616433]))


In [38]:

ind = np.where(D==1)[0]
M1 = M[ind]
W1 = W[ind]
X1 = X[ind,:]
Z1 = Z[ind]
Y1 = Y[ind]


A2 = trans.fit_transform(np.column_stack((M1,X1,Z1)))
A1 = trans.fit_transform(np.column_stack((M1,X1,W1)))

bridge_1 = splin_1.fit(A2, A1, Y1)


A1 = trans.fit_transform(np.column_stack((M,X,W)))
bridge_1_hat = bridge_1.predict(A1)
bridge_1_hat = bridge_1_hat.reshape(A1.shape[:1] + Y.shape[1:])
       
   
ind = np.where(D==0)[0]
W0 = W[ind]
X0 = X[ind,:]
Z0 = Z[ind]
bridge_1_hat = bridge_1_hat[ind]

B2 = trans.fit_transform(np.column_stack((Z0,X0)))
B1 = trans.fit_transform(np.column_stack((X0,W0)))


bridge_2 = splin_1.fit(B2, B1, bridge_1_hat)
       
gamma_0_hat = bridge_2.predict(trans.fit_transform(np.column_stack((X, W)))).reshape(-1, 1)

print(np.mean(gamma_0_hat))
print(np.var(gamma_0_hat))

3.9738505710350296
8.622606079442823


In [39]:
splin_2 = sparse2_l1vsl1(mu=0.1, V1=100, V2=100,
                 eta_alpha='auto', eta_w1='auto', eta_beta='auto', eta_w2='auto',
                 n_iter=10000, tol=.0001/2, sparsity=None, fit_intercept=True)


In [40]:
A = trans.fit_transform(np.column_stack((M,X,W)))
E = trans.fit_transform(np.column_stack((M,X,Z)))
B = trans.fit_transform(np.column_stack((X,W)))
C = trans.fit_transform(np.column_stack((X,Z)))

splin2_pred= splin_2.fit(A, B, C, E, Y, subsetted=True, subset_ind1=D).predict(B)
print(np.mean(splin2_pred))
print(np.var(splin2_pred))


3.9757064178481505
8.606148655305928


In [41]:
dml2_splin = DML_joint_mediated(Y, D, M, W, Z, X,
                 estimator='MR',
                 estimand='E[Y(1,M(0))]',
                 model1=splin_2, 
                 nn_1=False,
                 modelq1=splin_2, 
                 nn_q1=False,
                 n_folds=5, n_rep=1, opts={'lin_degree': deg})

print(dml2_splin.dml())


Rep: 1


100%|██████████| 5/5 [00:25<00:00,  5.00s/it]

(4.157383919162749, 16.810709837329757, array([3.977693  , 4.33707483]))
